In [1]:
# TODO, build some resistance against unseen actions into the thermal model
import sys
sys.path.append("..")
sys.path.append("../MPC")

from ControllerDataManager import ControllerDataManager
from ThermalModel import *
import pandas as pd 
import numpy as np
import pickle
import datetime
import yaml
import pytz

import matplotlib.pyplot as plt
from xbos import get_client

In [2]:
from TempThermalModel import ThermalModel

In [ ]:
zone = "HVAC_Zone_Large_Room"
params = []
all_data = dataManager.thermal_data(days_back=20)[zone]
end = all_data.index[-1]
start = all_data.index[0]
# Find best number of days to get for good thermalModel performance
for days_back in range(1, 102, 5):
    print("===============================")
    print("Getting %s number of days for thermal Data" % str(days_back))
    if end - datetime.timedelta(days=days_back) < start:
        print("Not enough data cached.")
        break
    data = all_data.loc[end - datetime.timedelta(days=days_back):end]
    thermalModel.fit(data, data['t_next'])
    for score_action in range(-1, 3):
        params.append(thermalModel._params)
        thermalModel.score(data, data['t_next'], scoreType=score_action)

In [ ]:
num_days = -4
for m_rmse, t_rmse, action, better in zip(thermalModel.model_error, thermalModel.baseline_error, thermalModel.scoreTypeList, thermalModel.betterThanBaseline):
    if action == -1:
        num_days += 5
    print("===============")
    print("num days", num_days)
    print("action", action)
    print("better?", better)
    print("model error", m_rmse)
    print("baseline error", t_rmse)
    
a_f = np.array(thermalModel.scoreTypeList) == -1
print(np.array(thermalModel.baseline_error)[a_f])



In [ ]:
# TODO instead of hardcoding input. just give it a dictionary it should append. 
def save_thermal_model_data(thermalModel, corresponding_days, params, file_name):
    """saves the whole model to a yaml file.
    RECOMMENDED: PYAML should be installed for prettier config file.
    param: thermalModel : object, thermal model which stores the types of scoring.
    param: corresponding_days: list, the order of data according to the number of days from current day.
    param: params, the past parameters"""
    config_dict = {}
    
    config_dict["Param Order"] = thermalModel._params_order
    config_dict["Params"] = params
    

    # store evaluations and RMSE's.
    config_dict["Evaluations"] = {}
    config_dict["Evaluations"]["Corresponding_Days"] = corresponding_days
    config_dict["Evaluations"]["Baseline"] = thermalModel.baseline_error
    config_dict["Evaluations"]["Model"] = thermalModel.model_error
    config_dict["Evaluations"]["ActionOrder"] = thermalModel.scoreTypeList
    config_dict["Evaluations"]["Better Than Baseline"] = thermalModel.betterThanBaseline

    with open("./thermal_model_" + file_name, 'wb') as ymlfile:
        # TODO Note import pyaml here to get a pretty config file.
        try:
            import pyaml
            pyaml.dump(config_dict, ymlfile)
        except ImportError:
            yaml.dump(config_dict, ymlfile)
            
days = sorted(4 * list(range(1, 101, 5)))
# TODO change file name
save_thermal_model_data(thermalModel, days, params, "HVAC_Zone_Large_Room" + "temp.yml")


In [12]:
# CONSISTANCY CHECKS

def check_consistency_for_cfg(cfg, days_back=50):
    print("===============Working on building %s===============" % cfg["Building"])
    c = get_client()

    dataManager = ControllerDataManager(cfg, c)
    thermalModel = ThermalModel()
    
    all_zone_data = dataManager.thermal_data(days_back=days_back)
    with open("./Eval_data/"+config["Building"], "wb") as f:
        import pickle
        pickle.dump(all_zone_data, f)
    
    for zone, data in all_zone_data.items():
        print("Checking consistency for zone %s" %zone)
        apply_consistency_check_to_data(data)
        

def apply_consistency_check_to_data(data):
    """
    :param data: Only used for fitting the thermal model."""
    # evaluate actions. On same temperatures, heating should increase, cooling decrease, and no action should be no different
    thermalModel = ThermalModel()
    thermalModel.fit(data, data["t_next"])

    def prepare_consistency_test_data(thermal_model, start_temperature=50, end_temperature=100, increments=5):
        filter_columns = thermal_model._filter_columns
        data = []
        for temperature in range(start_temperature, end_temperature+increments, increments):
            # TODO potentially not hardcode dt
                for dt in range(5,20, 5):
                     for action in range(0, 3):
                        datapoint = {"dt": dt, "a1": int(0 < action <= 1), "a2": int(1 < action <= 2),
                             "t_out": temperature, "t_in": temperature}
                        for col in filter_columns:
                            if "zone_temperature_" in col:
                                datapoint[col] = temperature
                        data.append(datapoint)
        return pd.DataFrame(data)

    consistancy_test_data = prepare_consistency_test_data(thermalModel)
    consistancy_test_data["prediction"] = thermalModel.predict(consistancy_test_data)

    def consistency_check(df):
        """Consistency check for a df with 3 entries. The datapoints can only differ in the action to meaningful.
        :param df: pd.df columns as given by ControllerDataManger plus a column with the predctions"""
        t_in = df['t_in'].values[0]
        dt = df["dt"].values[0]
        heating_temperature = df[df['a1'] == 1]["prediction"].values
        cooling_temperature = df[df['a2'] == 1]["prediction"].values
        no_action_temperature = df[(df['a2'] != 1) & (df['a1'] != 1)]["prediction"].values
        consistency_flag = True
        
        # TODO only use this check when t_out and zone temperature are the same as t_in
        # Following checks with t_in are only possible when everything has the same temperature
        # check if predicted heating temperature is higher than current
        if heating_temperature <= t_in:
            consistency_flag = False
            print("Warning, heating_temperature is lower than t_in.")
        if cooling_temperature >= t_in:
            consistency_flag = False
            print("Warning, cooling_temperature is higher than t_in.")
        
        
        # check that heating is more than no action and cooling
        if heating_temperature <= no_action_temperature or heating_temperature <= cooling_temperature:
            consistency_flag = False
            print("Warning, heating_temperature is too low compared to other actions.")
        # check cooling is lower than heating and no action
        if cooling_temperature >= no_action_temperature or cooling_temperature >= heating_temperature:
            consistency_flag = False
            print("Warning, cooling_temperature is too high compared to other actions.")
        # check if no action is between cooling and heating
        if not cooling_temperature < no_action_temperature < heating_temperature:
            consistency_flag = False
            print("Warning, no action is not inbetween heating temperature and cooling temperature.")
        
        # want to know for what data it didn't work
        if not consistency_flag:
            print("Inconsistency for following data:")
            print(df)
            print("")
        return consistency_flag



    consistancy_test_data.groupby(["t_in", "dt"]).apply(lambda df: consistency_check(df))






In [13]:
# CONSISTANCY CHECKS

import os
# Go to folder where all config files are located
config_folder_location = "../Buildings/"
all_dir = os.walk(config_folder_location).next()
for d in all_dir[1]:
    print("================================================")
    end_dir = config_folder_location + d + "/"

    files = os.walk(end_dir).next()[2]
    print("in dir: ", end_dir)
    for f in files:
        if ".yml" not in f:
            print("%s is not a yaml file. Continue to next file." % f)
            continue
        print("Getting file %s" % f)
        # Loads the configs
        with open("./" + end_dir + "/" + f, 'r') as o:
            config = yaml.load(o)
            
        check_consistency_for_cfg(config)
        


        
        


('in dir: ', '../Buildings/ciee/')
.DS_Store is not a yaml file. Continue to next file.
Getting file ciee.yml
===============Working on building ciee===============
Saw [xbos/hod] HodDB 1 seconds 726.08 ms ago
Saw [xbos/mdal] MDAL 6 seconds 882.197 ms ago
Saw [xbos/mdal] MDAL 3 seconds 55.075 ms ago
one zone preproccessed
one zone preproccessed
one zone preproccessed
one zone preproccessed
Checking consistency for zone HVAC_Zone_Eastzone
Warning, heating_temperature is higher than t_in.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
   a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_Centralzone  \
0   0   0   5    50     50                                      50   
1   1   0   5    50     50                                      50   
2   0   1   5    50     50                                      50   

   zone_temperature_HVAC_Zone_Northzone  zon

    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_Centralzone  \
57   0   0  10    80     80                                      80   
58   1   0  10    80     80                                      80   
59   0   1  10    80     80                                      80   

    zone_temperature_HVAC_Zone_Northzone  \
57                                    80   
58                                    80   
59                                    80   

    zone_temperature_HVAC_Zone_Southzone  prediction  
57                                    80        80.0  
58                                    80        80.4  
59                                    80        80.0  

Warning, heating_temperature is higher than t_in.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_Centralzone  \
63   0   0   

Warning, heating_temperature is too low compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_Centralzone  \
15   0   0  15    55     55                                      55   
16   1   0  15    55     55                                      55   
17   0   1  15    55     55                                      55   

    zone_temperature_HVAC_Zone_Eastzone  zone_temperature_HVAC_Zone_Northzone  \
15                                   55                                    55   
16                                   55                                    55   
17                                   55                                    55   

    prediction  
15       55.35  
16       55.35  
17       54.20  

Warning, heating_temperature is too low compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temp

Warning, heating_temperature is too low compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_Centralzone  \
72   0   0   5    90     90                                      90   
73   1   0   5    90     90                                      90   
74   0   1   5    90     90                                      90   

    zone_temperature_HVAC_Zone_Eastzone  zone_temperature_HVAC_Zone_Northzone  \
72                                   90                                    90   
73                                   90                                    90   
74                                   90                                    90   

    prediction  
72       90.10  
73       90.10  
74       89.75  

Warning, heating_temperature is too low compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temp

11                               55       55.05  

Warning, heating_temperature is higher than t_in.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_AC-1  \
12   0   0  10    55     55                               55   
13   1   0  10    55     55                               55   
14   0   1  10    55     55                               55   

    zone_temperature_HVAC_Zone_AC-3  prediction  
12                               55       55.10  
13                               55       55.55  
14                               55       55.10  

Warning, heating_temperature is higher than t_in.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_AC-1  \
15   0   0  15    55     55                               55   
16   1   0  15    55     55                   

...

Warning, heating_temperature is lower than t_in.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_Back_Hallway  \
81   0   0   5    95     95                                       95   
82   1   0   5    95     95                                       95   
83   0   1   5    95     95                                       95   

    zone_temperature_HVAC_Zone_Lobby  zone_temperature_HVAC_Zone_Main_Hallway  \
81                                95                                       95   
82                                95                                       95   
83                                95                                       95   

    zone_temperature_HVAC_Zone_Pegasus_Hall  \
81                                       95   
82                                       95   
83                                       95   

    zone_temperature_HVAC_Zone_Room_A  zone_temperature_HVAC_Zone_Room_D  \
81                                 95 

/Users/Daniel/BETS/.venv-bets/lib/python2.7/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Warning, heating_temperature is higher than t_in.
Warning, heating_temperature is too low compared to other actions.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
   a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_AC-2  \
0   0   0   5    50     50                               50   
1   1   0   5    50     50                               50   
2   0   1   5    50     50                               50   

   zone_temperature_HVAC_Zone_AC-3  prediction  
0                               50       50.05  
1                               50       50.40  
2                               50       55.05  

Warning, heating_temperature is higher than t_in.
Warning, heating_temperature is too low compared to other actions.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooli

Warning, heating_temperature is higher than t_in.
Warning, heating_temperature is too low compared to other actions.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_AC-2  \
45   0   0   5    75     75                               75   
46   1   0   5    75     75                               75   
47   0   1   5    75     75                               75   

    zone_temperature_HVAC_Zone_AC-3  prediction  
45                               75       75.05  
46                               75       75.40  
47                               75       80.05  

Warning, heating_temperature is higher than t_in.
Warning, heating_temperature is too low compared to other actions.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature a

    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_AC-2  \
78   0   0  15    90     90                               90   
79   1   0  15    90     90                               90   
80   0   1  15    90     90                               90   

    zone_temperature_HVAC_Zone_AC-3  prediction  
78                               90       90.20  
79                               90       91.25  
80                               90      105.20  

Warning, heating_temperature is higher than t_in.
Warning, heating_temperature is too low compared to other actions.
Warning, cooling_temperature is too high compared to other actions.
Warning, no action is not inbetween heating temperature and cooling temperature.
Inconsistency for following data:
    a1  a2  dt  t_in  t_out  zone_temperature_HVAC_Zone_AC-2  \
81   0   0   5    95     95                               95   
82   1   0   5    95     95                               95   
83   0   1   5    95     95                       

In [ ]:
thermal_model_zones = {zone: ThermalModel().fit(X=data, y=data["t_next"]) for zone, data in thermal_data.items()}

In [ ]:
def getBestHorizon(data):
    """Finds the best stretch of data, i.e. with most actions which are either heating or cooling. Gives
    preference to cooling.
    returns: best data slice"""
    horizon = datetime.timedelta(hours=3)
    start = datetime.datetime(year=2018, month=3, day=9, hour=8, minute=15)

    good_dates = []
    delta_minutes = datetime.timedelta(minutes=15)
    for row in data.itertuples():
        start = row[0]
        if sum(data.loc[start:start+horizon]["action"]) > 0:
            good_dates.append((start, sum(data.loc[start:start+horizon]["action"])))
    good_dates.sort(key=lambda x: x[1])
    new_start = good_dates[-1][0]
    return data.loc[new_start:new_start+horizon]

In [ ]:
trivial_predictions = {zone: np.ones(data.shape[0]) * data["t_in"][0] for zone, data in thermal_data.items()} # we predict the first temperature for the whole horizon


In [ ]:
def constantZonePredictions(data, thermal_model):
    zone_temp_filter = data.columns[["zone_temperature_" in col for col in data.columns]]
    constant_zone_temperatures = data[zone_temp_filter].iloc[0] # get the first row for constant zone temperatures. 

    constant_zone_predictions = [data.iloc[0]["t_in"]]
    for row in data.iterrows():
        # constant zone temperature predictions
        row_data = row[1] # work with the row data
        row_data["t_in"] = constant_zone_predictions[-1] # assign last prediction
        row_data[constant_zone_temperatures.index] = constant_zone_temperatures.values
        constant_zone_predictions.append(thermal_model.predict(row_data))
    return constant_zone_predictions

def perfectZonePredictions(data, thermal_model):
    perfect_zone_predictions = [data.iloc[0]["t_in"]]
    for row in data.iterrows():
        row_data = row[1] # work with the row data
        # assume we know the zone temperatures perfectly
        row_data["t_in"] = perfect_zone_predictions[-1] # assign last prediction
        perfect_zone_predictions.append(thermal_model.predict(row_data))
    return perfect_zone_predictions



In [ ]:
southzone_best_data = getBestHorizon(thermal_data["HVAC_Zone_Southzone"]) # use some interval
start = southzone_best_data.index[0]
end = southzone_best_data.index[-1]
bestData = {zone: data[start:end] for zone, data in thermal_data.items()}
rename_col = {zone: "zone_temperature_" + zone for zone in thermal_data.keys()}
for zone in thermal_data.keys():
    for zone1 in thermal_data.keys():
        ti = "zone_temperature_" + zone1
        if ti not in southzone_best_data.columns and zone != zone1:
            bestData[zone] = southzone_best_data.rename({"t_in" : ti, "zone_temperature_"+ zone: "t_in"}, axis='columns')

In [ ]:
print(bestData)

In [ ]:
# bestData = {zone: southzone_best_data for zone, data in thermal_data.items()}

# iteration evaluation
# start by assuming that we only know current zone temperatures. Find predictions for each zone that way
last_prediction_data = {zone: constantZonePredictions(data, thermal_model_zones[zone]) for zone, data in bestData.items()}
print(last_prediction_data)
for _ in range(1):
    temp_prediction_data = {}
    for zone, data in bestData.items():
        for prediction_zone, prediction in last_prediction_data.items():
            if prediction_zone != zone:
                data.loc[:,"zone_temperature_" + prediction_zone] = prediction[:-1] # TODO NOTE we predict one more than the data we have. 
        print(data)
        temp_prediction_data[zone] = perfectZonePredictions(data, thermal_model_zones[zone])
    last_prediction_data = temp_prediction_data
        
print(last_prediction_data)

In [ ]:
d = getBestHorizon(thermal_data["HVAC_Zone_Southzone"])
print(constantZonePredictions(d, thermal_model_zones["HVAC_Zone_Southzone"]))

print(trivial_prediction)
print(constant_zone_predictions)
print(perfect_zone_predictions)

In [ ]:
trivial_error = []
constant_error = []
perfect_error = []

dt = data["dt"]
t_in = data["t_in"]
for i in range(1,len(t_in)):
    trivial_error.append(thermal_model._normalizedRMSE_STD(trivial_prediction[:i], t_in[:i], dt[:i]))
    constant_error.append(thermal_model._normalizedRMSE_STD(constant_zone_predictions[:i], t_in[:i], dt[:i]))
    perfect_error.append(thermal_model._normalizedRMSE_STD(perfect_zone_predictions[:i], t_in[:i], dt[:i]))

In [ ]:
diff = constant_zone_predictions[:-1] - t_in[:]
print(np.square(diff))

plt.plot(trivial_error)
plt.show()

print(constant_error)
plt.plot(constant_error)
plt.show()

plt.plot(perfect_error)
plt.show()

In [ ]:
np.array(constant_error)[:, 1]

$$T_{in} + dt * ( a_1*c_1 + a_2*c_2 + (T_{out} - T_{in})*c_3 + c_4 + \sum_{i = 0}^N (T_{zone_i} - T_{in}) * c_{5+i})$$